# process_cmip6_files.ipynb
Post-processing of CMIP6 data that have been downloaded from ESGF:
1. Regrid to regular lon/lat grid (using CDO) and calculate statistics (e.g. CDO's yearmean).

B. S. Grandey, 2022

In [1]:
! date

Thu Feb 17 18:05:04 +08 2022


In [2]:
import contextlib
import pathlib
from cdo import Cdo, CDOException

In [3]:
cdo = Cdo()

print(f'CDO version: {cdo.version()}')
print(f'cdo.py bindings version: {cdo.__version__()}')

CDO version: 2.0.3
cdo.py bindings version: 1.5.4


## Base paths

In [4]:
# CMIP6 input data downloaded from ESGF (organised as <variable>/<source_id>_<member_id>/<instance_id>)
in_base = pathlib.Path('~/Data/p22b/CMIP6/').expanduser()
# Output data
out_base = pathlib.Path('~/Data/p22c/CMIP6/').expanduser()
out_base.mkdir(exist_ok=True)

## Function to regrid available data (using CDO) for a given variable, table_id, source-member pair, and experiment
Notes regarding available interpolation methods, after testing CDO's gencon2, gencon, and genbic on Omon zos data:
1. For unstructured grids, gencon works but not genbic ("Bilinear/bicubic interpolation doesn't support unstructured source grids!").
2. For curvilinear grids, genbic works. gencon/gencon2 mostly works, but fails for some grids (with most common error being "Source grid cell corner coordinates missing!"), and produces strange results for some others.
3. For lonlat grids, all methods work.

Based on the above observations, one possibility would be to prefer genbic for lonlat and curvilinear, and gencon for unstructured. If these fail, then other methods - including gennn (nearest neighbour) - can be used as fallback methods.

In [5]:
def regrid_using_cdo(variable='zos',
                     table_id='Omon',
                     source_member='ACCESS-CM2_r1i1p1f1',
                     experiment='historical',
                     stat_str='-yearmean',  # calculate statistical values after regridding?
                     force=False,  # overwrite previously produced files?
                    ):
    
    print(f'---- {variable} {table_id} {source_member} {experiment} {stat_str} ----')
    
    # List of pre-existing files skipped (assuming force=False)
    existing_list = []
    # List of output files written
    out_list = []
    
    # Search for available input directories, then identify most appropriate directory
    in_dirs = list(in_base.glob(f'{variable}/{source_member}/CMIP6.*.{experiment}.*.{table_id}.{variable}.*'))
    in_dirs = sorted(in_dirs)
    # Ignore directories with no NetCDF files
    for d in in_dirs:
        if len(list(d.glob("*.nc"))) == 0:
            in_dirs.remove(d)
    # Case: no input directories
    if len(in_dirs) == 0:
        print('No suitable input data found')
        return None  # exit (if no input directories)
    # Case: one suitable input directory
    elif len(in_dirs) == 1:
        in_dir = in_dirs[0]
        print(f'1 input dir: {in_dir.name} ({len(list(in_dir.glob("*.nc")))} nc file(s))')
    # Case: more than one suitable input directory
    else:
        print(f'{len(in_dirs)} input dirs:')
        for d in in_dirs:
            print(f'{d.name} ({len(list(d.glob("*.nc")))} nc file(s))')
        # Select most recently dated version
        vmax = 0  # initialise "max" version as 0
        for d in in_dirs:
            vmax = max(vmax, int(d.name[-8:]))
        for d in in_dirs:
            if str(vmax) not in d.name:
                in_dirs.remove(d)
        # After version selection, does only one input dir remain?
        if len(in_dirs) == 1:
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name} (most recent) ')
        # Prefer data that has already been remapped
        else:
            for d in in_dirs:
                if '.gn.' in d.name:
                    in_dirs.remove(d)
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name} (non-native grid)')
            
    # Is this a special case that requires additional pre-processing?
    # Special case 1: AWI-CM-1-1-MR thetaoga (which CDO fails to read)
    if (source_member.split('_')[0] == 'AWI-CM-1-1-MR') and (variable in ['thetaoga',]):
        print(f'Pre-processing {source_member} {variable} (remove nodes and ncells dims using ncwa)')
        # Output directory for pre-processing (special case 1)
        pre_dir = out_base.joinpath(f'regrid_pre/{variable}/{source_member}/{in_dir.name}')
        pre_dir.mkdir(parents=True, exist_ok=True)
        # Lists of pre-processing files that are skipped (already existing) and of files written
        skipped_pre_list = []
        written_pre_list = []
        # Loop over input data files (special case 1)
        in_fns = sorted(list(in_dir.glob('*.nc')))
        for in_fn in in_fns:
            # Pre-processing filename (special case 1)
            pre_fn = pre_dir.joinpath(f'{in_fn.stem}.ncwa.nc')
            # Do not overwrite, unless force=True (special case 1)
            if pre_fn.exists() and not force:
                skipped_pre_list.append(pre_fn.name)
                continue
            # Use NCO to remove nodes and ncells dimensions (special case 1)
            ! ncwa -a nodes,ncells {in_fn} {pre_fn}
            if pre_fn.exists():
                written_pre_list.append(pre_fn.name)
            else:
                print(f'Failed to write {pre_fn.name}')
        # Print summaries of existing files skipped and files written (special case 1)
        if len(skipped_pre_list) >= 1:
            print(f'Skipped {len(skipped_pre_list)} existing files, inc {skipped_pre_list[0]}')
        if len(written_pre_list) >= 1:
            print(f'Written {len(written_pre_list)} files, inc {written_pre_list[0]}')
        # Input directory is now pre_dir (special case 1)
        in_dir = pre_dir
    # Special case 2: IPSL-CM6A-LR_r1i1p1f1 2D ocean (CDO sees additional generic grid in some files)
    if (source_member == 'IPSL-CM6A-LR_r1i1p1f1') and (variable in ['zos', 'hfds', 'hfgeou', 'hfcorr']):
        print(f'Pre-processing {source_member} {experiment} {variable} (add area:coordinates attribute)')
        # Output directory for pre-processing (special case 2)
        pre_dir = out_base.joinpath(f'regrid_pre/{variable}/{source_member}/{in_dir.name}')
        pre_dir.mkdir(parents=True, exist_ok=True)
        # Lists of pre-processing files that are skipped (already existing) and of files written
        skipped_pre_list = []
        written_pre_list = []
        # Loop over input data files (special case 2)
        in_fns = sorted(list(in_dir.glob('*.nc')))
        for in_fn in in_fns:
            # Pre-processing filename (special case 2)
            pre_fn = pre_dir.joinpath(f'{in_fn.stem}.ncatted.nc')
            # Do not overwrite, unless force=True (special case 2)
            if pre_fn.exists() and not force:
                skipped_pre_list.append(pre_fn.name)
                continue
            # Use NCO to add area:coordinates attribute (special case 2)
            ! ncatted -a coordinates,area,o,c,"nav_lon nav_lat" {in_fn} {pre_fn}
            if pre_fn.exists():
                written_pre_list.append(pre_fn.name)
            else:
                print(f'Failed to write {pre_fn.name}')
        # Print summaries of existing files skipped and files written (special case 2)
        if len(skipped_pre_list) >= 1:
            print(f'Skipped {len(skipped_pre_list)} existing files, inc {skipped_pre_list[0]}')
        if len(written_pre_list) >= 1:
            print(f'Written {len(written_pre_list)} files, inc {written_pre_list[0]}')
        # Input directory is now pre_dir (special case 2)
        in_dir = pre_dir
    
    # Identify input data files
    in_fns = sorted(list(in_dir.glob('*.nc')))
    # Input grid type
    try:
        with contextlib.redirect_stdout(None):  # suppress CDO error message details (if any)
            grid_str = [s for s in cdo.sinfo(input=f'{in_fns[0]}') if 'points' in s][0]
        in_grid_type = grid_str.split(':')[1].strip()
        in_grid_points = grid_str.split(':')[-1].strip()
        print(f'Input data grid type is {in_grid_type} ({in_grid_points})')
    except CDOException:  # if Globus transfer is in progress, incomplete files may be encountered
        print(f'Failed to read {in_fns[0].name} - is it currently being transferred?')
        return None  # exit (if incomplete file encountered)
    # Output data directory
    if stat_str:
        out_dir = out_base.joinpath(f'regrid_{stat_str.strip("-")}/{variable}/{source_member}/{in_dir.name}')
    else:
        out_dir = out_base.joinpath(f'regrid/{variable}/{source_member}/{in_dir.name}')
    out_dir.mkdir(parents=True, exist_ok=True)
    
    # If points=1 (e.g. due to already being a global mean), then just reduce dimensions
    if in_grid_points in ['points=1', 'points=1 (1x1)']:
        print('Reduce dimensions')
        # Loop over in_fns (if points=1)
        for in_fn in in_fns:
            # Output filename (if points=1)
            if stat_str:
                out_fn = out_dir.joinpath(f'{in_fn.stem}.1d.{stat_str.strip("-")}.nc')
            else:
                out_fn = out_dir.joinpath(f'{in_fn.stem}.1d.nc')
            # Do not overwrite, unless force=True (if points=1)
            if out_fn.exists() and not force:
                existing_list.append(out_fn.name)
                continue
            # Write output file (if points=1)
            cdo.selname(variable, input=f'{stat_str} {in_fn}', output=f'{out_fn}',
                        options='--reduce_dim')
            if out_fn.exists():
                out_list.append(out_fn.name)
            else:
                print(f'Failed to write {out_fn.name}')
        # Print summary of existing files skipped (if points=1)
        if len(existing_list) == 1:
            print(f'Skipped existing file: {existing_list[0]}')
        elif len(existing_list) > 1:
            print(f'Skipped {len(existing_list)} existing files, inc {existing_list[0]}')
        # Print summary of files written (if points=1)
        if len(out_list) == 1:
            print(f'Written {out_list[0]}')
        elif len(out_list) > 1:
            print(f'Written {len(out_list)} files, inc {out_list[0]}')
        return out_list  # exit (if points=1)
    
    # Preferred regridding methods depend on input grid type
    # Curvilinear: prefer bicubic. (Issues with gencon2 and gencon for some CMIP6 curvilinear grids.)
    # Lonlat, gaussian: same preference as for curvilinear.
    if in_grid_type in ['lonlat', 'curvilinear', 'gaussian']:
        preferred_methods=['genbic', 'gencon', 'gennn', None]
    # Unstructured: prefer 1st order conservative. (Issues with 2nd order conservative for unstructured.)
    elif in_grid_type in ['unstructured',]:
        preferred_methods=['gencon', 'genbic', 'gennn', None]
    else:
        preferred_methods=['gencon', 'genbic', 'gennn', None]
    # Weights data directory
    weights_dir = out_base.joinpath(f'regrid_weights/{variable}/{source_member}/{in_dir.name}')
    weights_dir.mkdir(parents=True, exist_ok=True)
    # Generate regridding weights file
    for method in preferred_methods:
        print(f'Trying {method}')
        weights_fn = weights_dir.joinpath(f'{in_fns[0].stem}.{method}_weights.nc')
        # If all preferred methods have been exhausted, then exit
        if method == None:
            return None  # exit
        # If weights file already exists, use pre-existing file (unless force=True)
        if weights_fn.exists() and not force:
            print(f'{weights_fn.name} already exists')
            break  # if weights file exists, no need to try other methods
        # Otherwise, try calculating weights and writing file
        else:
            try:
                with contextlib.redirect_stdout(None):  # suppress CDO error message details
                    if method == 'gencon2':
                        cdo_res = cdo.gencon2('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    elif method == 'gencon':
                        cdo_res = cdo.gencon('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')          
                    elif method == 'genbic':
                        cdo_res = cdo.genbic('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    elif method == 'gennn':
                        cdo_res = cdo.gennn('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    else:
                        continue
                if weights_fn.exists():
                    print(f'Written {weights_fn.name}')
                    break  # if file written successfully, then no need to try other methods
            except CDOException:
                print(f'Failed to write {weights_fn.name}')
    
    # Loop over input data files and regrid
    print(f'{len(in_fns)} input data file(s) to regrid')
    for in_fn in in_fns:
        # Output data filename
        if stat_str:
            out_fn = out_dir.joinpath(f'{in_fn.stem}.{method[3:]}.{stat_str.strip("-")}.nc')
        else:
            out_fn = out_dir.joinpath(f'{in_fn.stem}.{method[3:]}.nc')
        # If output file already exists, do not overwrite (unless force=True)
        if out_fn.exists() and not force:
            existing_list.append(out_fn.name)
            continue
        # Regrid using CDO
        cdo_res = cdo.sellonlatbox(0, 360, -90, 90,  # shift longitudes
                                   input=f'{stat_str} -selname,{variable} -remap,global_1,{weights_fn} {in_fn}',
                                   output=f'{out_fn}')
        # Check if written
        if out_fn.exists():
            out_list.append(out_fn.name)
        else:
            print(f'Failed to write {out_fn.name}')
    
    # Print summary of existing files skipped
    if len(existing_list) == 1:
        print(f'Skipped existing file: {existing_list[0]}')
    elif len(existing_list) > 1:
        print(f'Skipped {len(existing_list)} existing files, inc {existing_list[0]}')
    # Print summary of files written (if points=1)
    if len(out_list) == 1:
        print(f'Written {out_list[0]}')
    elif len(out_list) > 1:
        print(f'Written {len(out_list)} files, inc {out_list[0]}')
    
    return out_list  # exit

## Apply function(s) to variables and experiments of interest

### Annual means (yearmean) of regridded Omon data

In [6]:
%%time
# For variables and experiments of interest, apply regridding function to all available source-member pairs
# Loop over variables of interest
table_id = 'Omon'
for variable in ['zostoga', 'thetaoga',  # 1D ocean
                 'zos', 'hfds', 'hfgeou', 'hfcorr']:  # 2D ocean
    # Identify and loop over available source-member pairs
    source_member_pairs = sorted([d.name for d in in_base.glob(f'{variable}/[!.]*_*')])
    for source_member in source_member_pairs:
        # Loop over experiments of interest - include piControl for Omon
        for experiment in ['piControl', 'historical', 'ssp585', 'ssp370', 'ssp245', 'ssp126']:
            # Regrid
            temp = regrid_using_cdo(variable=variable,
                                    table_id=table_id,
                                    source_member=source_member,
                                    experiment=experiment,
                                    stat_str='-yearmean',
                                    force=False)
        print('', flush=True)  # empty line between source-member pairs

---- zostoga Omon ACCESS-CM2_r1i1p1f1 piControl -yearmean ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.piControl.r1i1p1f1.Omon.zostoga.gn.v20191112 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_ACCESS-CM2_piControl_r1i1p1f1_gn_095001-144912.1d.yearmean.nc
---- zostoga Omon ACCESS-CM2_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.r1i1p1f1.Omon.zostoga.gn.v20191108 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_ACCESS-CM2_historical_r1i1p1f1_gn_185001-201412.1d.yearmean.nc
---- zostoga Omon ACCESS-CM2_r1i1p1f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CSIRO-ARCCSS.ACCESS-CM2.ssp585.r1i1p1f1.Omon.zostoga.gn.v20210317 (2 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped 2 existing files, inc zostoga_Omon_ACCESS-CM2_ssp585_r1i1p1f1_gn_201501-210012.1d.yearmean.nc
--

Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_CMCC-ESM2_historical_r1i1p1f1_gn_185001-201412.1d.yearmean.nc
---- zostoga Omon CMCC-ESM2_r1i1p1f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CMCC.CMCC-ESM2.ssp585.r1i1p1f1.Omon.zostoga.gn.v20210126 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_CMCC-ESM2_ssp585_r1i1p1f1_gn_201501-210012.1d.yearmean.nc
---- zostoga Omon CMCC-ESM2_r1i1p1f1 ssp370 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CMCC.CMCC-ESM2.ssp370.r1i1p1f1.Omon.zostoga.gn.v20210202 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_CMCC-ESM2_ssp370_r1i1p1f1_gn_201501-210012.1d.yearmean.nc
---- zostoga Omon CMCC-ESM2_r1i1p1f1 ssp245 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CMCC.CMCC-ESM2.ssp245.r1i1p1f1.Omon.zostoga.gn.v20210129 (1 nc file(s))
Input data grid type is generic (points=1)
Re

Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_CanESM5-CanOE_ssp245_r1i1p2f1_gn_201501-210012.1d.yearmean.nc
---- zostoga Omon CanESM5-CanOE_r1i1p2f1 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CCCma.CanESM5-CanOE.ssp126.r1i1p2f1.Omon.zostoga.gn.v20190429 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_CanESM5-CanOE_ssp126_r1i1p2f1_gn_201501-210012.1d.yearmean.nc

---- zostoga Omon CanESM5_r1i1p1f1 piControl -yearmean ----
1 input dir: CMIP6.CMIP.CCCma.CanESM5.piControl.r1i1p1f1.Omon.zostoga.gn.v20190429 (5 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped 5 existing files, inc zostoga_Omon_CanESM5_piControl_r1i1p1f1_gn_520101-540012.1d.yearmean.nc
---- zostoga Omon CanESM5_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.CCCma.CanESM5.historical.r1i1p1f1.Omon.zostoga.gn.v20190429 (1 nc file(s))
Input data grid type is gener

Input data grid type is generic (points=1)
Reduce dimensions
Skipped 86 existing files, inc zostoga_Omon_EC-Earth3-Veg-LR_ssp585_r1i1p1f1_gn_201501-201512.1d.yearmean.nc
---- zostoga Omon EC-Earth3-Veg-LR_r1i1p1f1 ssp370 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg-LR.ssp370.r1i1p1f1.Omon.zostoga.gn.v20201123 (86 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped 86 existing files, inc zostoga_Omon_EC-Earth3-Veg-LR_ssp370_r1i1p1f1_gn_201501-201512.1d.yearmean.nc
---- zostoga Omon EC-Earth3-Veg-LR_r1i1p1f1 ssp245 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg-LR.ssp245.r1i1p1f1.Omon.zostoga.gn.v20201123 (86 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped 86 existing files, inc zostoga_Omon_EC-Earth3-Veg-LR_ssp245_r1i1p1f1_gn_201501-201512.1d.yearmean.nc
---- zostoga Omon EC-Earth3-Veg-LR_r1i1p1f1 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.EC-Earth-

Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_GISS-E2-1-G_ssp370_r1i1p1f2_gn_201501-210012.1d.yearmean.nc
---- zostoga Omon GISS-E2-1-G_r1i1p1f2 ssp245 -yearmean ----
No suitable input data found
---- zostoga Omon GISS-E2-1-G_r1i1p1f2 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.NASA-GISS.GISS-E2-1-G.ssp126.r1i1p1f2.Omon.zostoga.gn.v20200115 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_GISS-E2-1-G_ssp126_r1i1p1f2_gn_201501-210012.1d.yearmean.nc

---- zostoga Omon GISS-E2-1-G_r1i1p3f1 piControl -yearmean ----
No suitable input data found
---- zostoga Omon GISS-E2-1-G_r1i1p3f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.NASA-GISS.GISS-E2-1-G.historical.r1i1p3f1.Omon.zostoga.gn.v20190702 (1 nc file(s))
Input data grid type is generic (points=1)
Reduce dimensions
Skipped existing file: zostoga_Omon_GISS-E2-1-G_historical_r1i1p3f1_gn_185001-201412.1d.yearmean.n

...

Input data grid type is curvilinear (points=105704 (362x292))
Trying genbic
hfds_Omon_NESM3_ssp126_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: hfds_Omon_NESM3_ssp126_r1i1p1f1_gn_201501-210012.bic.yearmean.nc

---- hfds Omon NorESM2-LM_r1i1p1f1 piControl -yearmean ----
1 input dir: CMIP6.CMIP.NCC.NorESM2-LM.piControl.r1i1p1f1.Omon.hfds.gn.v20210118 (51 nc file(s))
Input data grid type is curvilinear (points=138600 (360x385))
Trying genbic
hfds_Omon_NorESM2-LM_piControl_r1i1p1f1_gn_160001-160912.genbic_weights.nc already exists
51 input data file(s) to regrid
Skipped 51 existing files, inc hfds_Omon_NorESM2-LM_piControl_r1i1p1f1_gn_160001-160912.bic.yearmean.nc
---- hfds Omon NorESM2-LM_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.NCC.NorESM2-LM.historical.r1i1p1f1.Omon.hfds.gn.v20190815 (17 nc file(s))
Input data grid type is curvilinear (points=138600 (360x385))
Trying genbic
hfds_Omon_NorESM2-LM_histori

---- hfds Omon UKESM1-0-LL_r1i1p1f2 piControl -yearmean ----
1 input dir: CMIP6.CMIP.MOHC.UKESM1-0-LL.piControl.r1i1p1f2.Omon.hfds.gn.v20200828 (20 nc file(s))
Input data grid type is curvilinear (points=118800 (360x330))
Trying genbic
hfds_Omon_UKESM1-0-LL_piControl_r1i1p1f2_gn_196001-204912.genbic_weights.nc already exists
20 input data file(s) to regrid
Skipped 20 existing files, inc hfds_Omon_UKESM1-0-LL_piControl_r1i1p1f2_gn_196001-204912.bic.yearmean.nc
---- hfds Omon UKESM1-0-LL_r1i1p1f2 historical -yearmean ----
2 input dirs:
CMIP6.CMIP.MOHC.UKESM1-0-LL.historical.r1i1p1f2.Omon.hfds.gn.v20190406 (2 nc file(s))
CMIP6.CMIP.MOHC.UKESM1-0-LL.historical.r1i1p1f2.Omon.hfds.gn.v20190627 (2 nc file(s))
Using CMIP6.CMIP.MOHC.UKESM1-0-LL.historical.r1i1p1f2.Omon.hfds.gn.v20190627 (most recent) 
Input data grid type is curvilinear (points=118800 (360x330))
Trying genbic
hfds_Omon_UKESM1-0-LL_historical_r1i1p1f2_gn_185001-194912.genbic_weights.nc already exists
2 input data file(s) to regr

Input data grid type is curvilinear (points=105704 (362x292))
Trying genbic
hfgeou_Omon_EC-Earth3-Veg_ssp370_r1i1p1f1_gn_201501-201512.genbic_weights.nc already exists
86 input data file(s) to regrid
Skipped 86 existing files, inc hfgeou_Omon_EC-Earth3-Veg_ssp370_r1i1p1f1_gn_201501-201512.bic.yearmean.nc
---- hfgeou Omon EC-Earth3-Veg_r1i1p1f1 ssp245 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg.ssp245.r1i1p1f1.Omon.hfgeou.gn.v20200919 (86 nc file(s))
Input data grid type is curvilinear (points=105704 (362x292))
Trying genbic
hfgeou_Omon_EC-Earth3-Veg_ssp245_r1i1p1f1_gn_201501-201512.genbic_weights.nc already exists
86 input data file(s) to regrid
Skipped 86 existing files, inc hfgeou_Omon_EC-Earth3-Veg_ssp245_r1i1p1f1_gn_201501-201512.bic.yearmean.nc
---- hfgeou Omon EC-Earth3-Veg_r1i1p1f1 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-Veg.ssp126.r1i1p1f1.Omon.hfgeou.gn.v20200919 (86 nc file(s))
Input data grid type

Written hfcorr_Omon_EC-Earth3-CC_piControl_r1i1p1f1_gn_185001-185012.genbic_weights.nc
505 input data file(s) to regrid
Written 505 files, inc hfcorr_Omon_EC-Earth3-CC_piControl_r1i1p1f1_gn_185001-185012.bic.yearmean.nc
---- hfcorr Omon EC-Earth3-CC_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.EC-Earth-Consortium.EC-Earth3-CC.historical.r1i1p1f1.Omon.hfcorr.gn.v20210113 (165 nc file(s))
Input data grid type is curvilinear (points=105704 (362x292))
Trying genbic
Written hfcorr_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185001-185012.genbic_weights.nc
165 input data file(s) to regrid
Written 165 files, inc hfcorr_Omon_EC-Earth3-CC_historical_r1i1p1f1_gn_185001-185012.bic.yearmean.nc
---- hfcorr Omon EC-Earth3-CC_r1i1p1f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.EC-Earth-Consortium.EC-Earth3-CC.ssp585.r1i1p1f1.Omon.hfcorr.gn.v20210113 (86 nc file(s))
Input data grid type is curvilinear (points=105704 (362x292))
Trying genbic
Written hfcorr_Omon_EC-Earth3-CC_ssp585_

Input data grid type is curvilinear (points=130680 (360x363))
Trying genbic
Written hfcorr_Omon_MRI-ESM2-0_ssp126_r1i1p1f1_gn_201501-210012.genbic_weights.nc
2 input data file(s) to regrid
Written 2 files, inc hfcorr_Omon_MRI-ESM2-0_ssp126_r1i1p1f1_gn_201501-210012.bic.yearmean.nc

CPU times: user 13.3 s, sys: 48.8 s, total: 1min 2s
Wall time: 24min 11s


### Annual means (yearmean) of regridded Amon data

In [7]:
%%time
# For variables and experiments of interest, apply regridding function to all available source-member pairs
# Loop over variables of interest
table_id = 'Amon'
for variable in ['rlut', 'rsdt', 'rsut', 'tas']:
    # Identify and loop over available source-member pairs
    source_member_pairs = sorted([d.name for d in in_base.glob(f'{variable}/[!.]*_*')])
    for source_member in source_member_pairs:
        # Loop over experiments of interest - do not include piControl for Amon
        for experiment in ['piControl', 'historical', 'ssp585', 'ssp370', 'ssp245', 'ssp126']:
            # Regrid
            temp = regrid_using_cdo(variable=variable,
                                    table_id=table_id,
                                    source_member=source_member,
                                    experiment=experiment,
                                    stat_str='-yearmean',
                                    force=False)
        print('', flush=True)  # empty line between source-member pairs

---- rlut Amon ACCESS-CM2_r1i1p1f1 piControl -yearmean ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.piControl.r1i1p1f1.Amon.rlut.gn.v20191112 (1 nc file(s))
Input data grid type is lonlat (points=27648 (192x144))
Trying genbic
rlut_Amon_ACCESS-CM2_piControl_r1i1p1f1_gn_095001-144912.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_ACCESS-CM2_piControl_r1i1p1f1_gn_095001-144912.bic.yearmean.nc
---- rlut Amon ACCESS-CM2_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.r1i1p1f1.Amon.rlut.gn.v20191108 (1 nc file(s))
Input data grid type is lonlat (points=27648 (192x144))
Trying genbic
rlut_Amon_ACCESS-CM2_historical_r1i1p1f1_gn_185001-201412.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_ACCESS-CM2_historical_r1i1p1f1_gn_185001-201412.bic.yearmean.nc
---- rlut Amon ACCESS-CM2_r1i1p1f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CSIR

---- rlut Amon BCC-CSM2-MR_r1i1p1f1 piControl -yearmean ----
1 input dir: CMIP6.CMIP.BCC.BCC-CSM2-MR.piControl.r1i1p1f1.Amon.rlut.gn.v20181016 (1 nc file(s))
Input data grid type is gaussian (points=51200 (320x160)  F80)
Trying genbic
rlut_Amon_BCC-CSM2-MR_piControl_r1i1p1f1_gn_185001-244912.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_BCC-CSM2-MR_piControl_r1i1p1f1_gn_185001-244912.bic.yearmean.nc
---- rlut Amon BCC-CSM2-MR_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.BCC.BCC-CSM2-MR.historical.r1i1p1f1.Amon.rlut.gn.v20181126 (1 nc file(s))
Input data grid type is gaussian (points=51200 (320x160)  F80)
Trying genbic
rlut_Amon_BCC-CSM2-MR_historical_r1i1p1f1_gn_185001-201412.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_BCC-CSM2-MR_historical_r1i1p1f1_gn_185001-201412.bic.yearmean.nc
---- rlut Amon BCC-CSM2-MR_r1i1p1f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP

Input data grid type is lonlat (points=32768 (256x128))
Trying genbic
rlut_Amon_CAS-ESM2-0_ssp370_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_CAS-ESM2-0_ssp370_r1i1p1f1_gn_201501-210012.bic.yearmean.nc
---- rlut Amon CAS-ESM2-0_r1i1p1f1 ssp245 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CAS.CAS-ESM2-0.ssp245.r1i1p1f1.Amon.rlut.gn.v20201228 (1 nc file(s))
Input data grid type is lonlat (points=32768 (256x128))
Trying genbic
rlut_Amon_CAS-ESM2-0_ssp245_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_CAS-ESM2-0_ssp245_r1i1p1f1_gn_201501-210012.bic.yearmean.nc
---- rlut Amon CAS-ESM2-0_r1i1p1f1 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CAS.CAS-ESM2-0.ssp126.r1i1p1f1.Amon.rlut.gn.v20201228 (1 nc file(s))
Input data grid type is lonlat (points=32768 (256x128))
Trying genbic
rlut_Amon_CAS-ESM2-0_ssp126_r1i1p1f1_gn_201501-210012.

Input data grid type is lonlat (points=55296 (288x192))
Trying genbic
rlut_Amon_CMCC-CM2-SR5_ssp585_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_CMCC-CM2-SR5_ssp585_r1i1p1f1_gn_201501-210012.bic.yearmean.nc
---- rlut Amon CMCC-CM2-SR5_r1i1p1f1 ssp370 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CMCC.CMCC-CM2-SR5.ssp370.r1i1p1f1.Amon.rlut.gn.v20200622 (1 nc file(s))
Input data grid type is lonlat (points=55296 (288x192))
Trying genbic
rlut_Amon_CMCC-CM2-SR5_ssp370_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_CMCC-CM2-SR5_ssp370_r1i1p1f1_gn_201501-210012.bic.yearmean.nc
---- rlut Amon CMCC-CM2-SR5_r1i1p1f1 ssp245 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CMCC.CMCC-CM2-SR5.ssp245.r1i1p1f1.Amon.rlut.gn.v20200617 (1 nc file(s))
Input data grid type is lonlat (points=55296 (288x192))
Trying genbic
rlut_Amon_CMCC-CM2-SR5_ssp245_r1i1p1

Input data grid type is gaussian (points=32768 (256x128)  F64)
Trying genbic
rlut_Amon_CNRM-CM6-1_ssp370_r1i1p1f2_gr_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_CNRM-CM6-1_ssp370_r1i1p1f2_gr_201501-210012.bic.yearmean.nc
---- rlut Amon CNRM-CM6-1_r1i1p1f2 ssp245 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-CM6-1.ssp245.r1i1p1f2.Amon.rlut.gr.v20190219 (1 nc file(s))
Input data grid type is gaussian (points=32768 (256x128)  F64)
Trying genbic
rlut_Amon_CNRM-CM6-1_ssp245_r1i1p1f2_gr_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: rlut_Amon_CNRM-CM6-1_ssp245_r1i1p1f2_gr_201501-210012.bic.yearmean.nc
---- rlut Amon CNRM-CM6-1_r1i1p1f2 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-CM6-1.ssp126.r1i1p1f2.Amon.rlut.gr.v20190219 (1 nc file(s))
Input data grid type is gaussian (points=32768 (256x128)  F64)
Trying genbic
rlut_Amon_CNRM-

...

Input data grid type is lonlat (points=21600 (180x120))
Trying genbic
tas_Amon_INM-CM5-0_ssp585_r1i1p1f1_gr1_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: tas_Amon_INM-CM5-0_ssp585_r1i1p1f1_gr1_201501-210012.bic.yearmean.nc
---- tas Amon INM-CM5-0_r1i1p1f1 ssp370 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.INM.INM-CM5-0.ssp370.r1i1p1f1.Amon.tas.gr1.v20190618 (1 nc file(s))
Input data grid type is lonlat (points=21600 (180x120))
Trying genbic
tas_Amon_INM-CM5-0_ssp370_r1i1p1f1_gr1_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: tas_Amon_INM-CM5-0_ssp370_r1i1p1f1_gr1_201501-210012.bic.yearmean.nc
---- tas Amon INM-CM5-0_r1i1p1f1 ssp245 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.INM.INM-CM5-0.ssp245.r1i1p1f1.Amon.tas.gr1.v20190619 (1 nc file(s))
Input data grid type is lonlat (points=21600 (180x120))
Trying genbic
tas_Amon_INM-CM5-0_ssp245_r1i1p1f1_gr1_201501-210012.genbic_weig

Input data grid type is lonlat (points=27648 (192x144))
Trying genbic
tas_Amon_KACE-1-0-G_ssp245_r1i1p1f1_gr_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: tas_Amon_KACE-1-0-G_ssp245_r1i1p1f1_gr_201501-210012.bic.yearmean.nc
---- tas Amon KACE-1-0-G_r1i1p1f1 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.NIMS-KMA.KACE-1-0-G.ssp126.r1i1p1f1.Amon.tas.gr.v20191007 (1 nc file(s))
Input data grid type is lonlat (points=27648 (192x144))
Trying genbic
tas_Amon_KACE-1-0-G_ssp126_r1i1p1f1_gr_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: tas_Amon_KACE-1-0-G_ssp126_r1i1p1f1_gr_201501-210012.bic.yearmean.nc

---- tas Amon KIOST-ESM_r1i1p1f1 piControl -yearmean ----
2 input dirs:
CMIP6.CMIP.KIOST.KIOST-ESM.piControl.r1i1p1f1.Amon.tas.gr1.v20191106 (1 nc file(s))
CMIP6.CMIP.KIOST.KIOST-ESM.piControl.r1i1p1f1.Amon.tas.gr1.v20210329 (1 nc file(s))
Using CMIP6.CMIP.KIOST.KIOST-ESM.piContro

Input data grid type is gaussian (points=32768 (256x128)  F64)
Trying genbic
tas_Amon_MIROC6_ssp245_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: tas_Amon_MIROC6_ssp245_r1i1p1f1_gn_201501-210012.bic.yearmean.nc
---- tas Amon MIROC6_r1i1p1f1 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.MIROC.MIROC6.ssp126.r1i1p1f1.Amon.tas.gn.v20190627 (1 nc file(s))
Input data grid type is gaussian (points=32768 (256x128)  F64)
Trying genbic
tas_Amon_MIROC6_ssp126_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: tas_Amon_MIROC6_ssp126_r1i1p1f1_gn_201501-210012.bic.yearmean.nc

---- tas Amon MPI-ESM-1-2-HAM_r1i1p1f1 piControl -yearmean ----
1 input dir: CMIP6.CMIP.HAMMOZ-Consortium.MPI-ESM-1-2-HAM.piControl.r1i1p1f1.Amon.tas.gn.v20200120 (50 nc file(s))
Input data grid type is gaussian (points=18432 (192x96)  F48)
Trying genbic
tas_Amon_MPI-ESM-1-2-HAM_piControl_r1i1

Input data grid type is gaussian (points=51200 (320x160)  F80)
Trying genbic
tas_Amon_MRI-ESM2-0_historical_r1i1p1f1_gn_185001-201412.genbic_weights.nc already exists
1 input data file(s) to regrid
Skipped existing file: tas_Amon_MRI-ESM2-0_historical_r1i1p1f1_gn_185001-201412.bic.yearmean.nc
---- tas Amon MRI-ESM2-0_r1i1p1f1 ssp585 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp585.r1i1p1f1.Amon.tas.gn.v20191108 (2 nc file(s))
Input data grid type is gaussian (points=51200 (320x160)  F80)
Trying genbic
tas_Amon_MRI-ESM2-0_ssp585_r1i1p1f1_gn_201501-210012.genbic_weights.nc already exists
2 input data file(s) to regrid
Skipped 2 existing files, inc tas_Amon_MRI-ESM2-0_ssp585_r1i1p1f1_gn_201501-210012.bic.yearmean.nc
---- tas Amon MRI-ESM2-0_r1i1p1f1 ssp370 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp370.r1i1p1f1.Amon.tas.gn.v20190222 (1 nc file(s))
Input data grid type is gaussian (points=51200 (320x160)  F80)
Trying genbic
tas_Amon_MRI-ESM2-0_ssp370

Input data grid type is lonlat (points=55296 (288x192))
Trying genbic
tas_Amon_NorESM2-MM_ssp370_r1i1p1f1_gn_201501-202012.genbic_weights.nc already exists
9 input data file(s) to regrid
Skipped 9 existing files, inc tas_Amon_NorESM2-MM_ssp370_r1i1p1f1_gn_201501-202012.bic.yearmean.nc
---- tas Amon NorESM2-MM_r1i1p1f1 ssp245 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.NCC.NorESM2-MM.ssp245.r1i1p1f1.Amon.tas.gn.v20191108 (9 nc file(s))
Input data grid type is lonlat (points=55296 (288x192))
Trying genbic
tas_Amon_NorESM2-MM_ssp245_r1i1p1f1_gn_201501-202012.genbic_weights.nc already exists
9 input data file(s) to regrid
Skipped 9 existing files, inc tas_Amon_NorESM2-MM_ssp245_r1i1p1f1_gn_201501-202012.bic.yearmean.nc
---- tas Amon NorESM2-MM_r1i1p1f1 ssp126 -yearmean ----
1 input dir: CMIP6.ScenarioMIP.NCC.NorESM2-MM.ssp126.r1i1p1f1.Amon.tas.gn.v20191108 (9 nc file(s))
Input data grid type is lonlat (points=55296 (288x192))
Trying genbic
tas_Amon_NorESM2-MM_ssp126_r1i1p1f1_gn_201501-20

In [8]:
! date

Thu Feb 17 18:47:10 +08 2022
